In [1]:
import numpy as np
import pandas as pd

from py.consts import DATA_DIR, DAYS_OF_WEEK, MONTHS_OF_YEAR
from py.utils import one_hot_encode

In [4]:
df = pd.read_pickle(DATA_DIR / "flitsers.pkl")

sample_size: int = 1000
print(f"number of records: {len(df)}")
df = df.sample(sample_size)
print(f"sampled to {sample_size} records")
    
df.head()

number of records: 96757
sampled to 1000 records


,id,datum,soort_weg,wegnummer,zijde,hm_paal,type_controle,tijd_van_melden,details,laatste_activiteit,...,weer_temp_min,weer_luchtdruk_hpa,weer_luchtvochtigheid_procent,weer_windsnelheid_m_per_sec,weer_windrichting_graden,weer_bewolking_procent,weer_regen_mm,weer_sneeuw_mm,weer_zonnestand,weer_locatie_naam
11373,96207,2022-05-05,nweg,N343,beide,26.6,Radar,10:33:55,\n</td,15:50:05.942842,...,13.99,1024.0,57.0,0.89,202.0,100.0,NaN,NaN,na_zonsopgang,Westerhaar-Vriezenveensewijk
21883,84345,2021-07-11,snelweg,A2,rechts,185.2,Radar,19:45:05,bij afslag budel onder viaduct\n</td,20:10:45.387596,...,20.95,1014.0,60.0,1.54,240.0,81.0,NaN,NaN,na_zonsopgang,Cranendonck
55158,42266,2019-01-12,snelweg,A28,rechts,91.2,Radar,07:43:00,voor afslag zwolle-zuid\r\n,17:15:02.972019,...,6.00,1020.0,87.0,4.60,230.0,90.0,NaN,NaN,voor_zonsopgang,Katerveer
52675,44764,2019-05-09,nweg,N203,beide,54.4,Laser,09:13:00,bij afslag naar de jachthaven vlakbij de a9\n...,11:50:39.597594,...,10.00,995.0,81.0,5.10,180.0,0.0,NaN,NaN,na_zonsopgang,Uitgeest
84278,13094,2017-04-19,snelweg,A12,rechts,65.4,Radar,14:22:41,bij texaco tankstation de forten\r\n,13:45:03.401523,...,8.00,1037.0,45.0,4.60,50.0,20.0,NaN,NaN,na_zonsopgang,Bunnik


In [5]:
df.shape

(1000, 27)

In [6]:
dataset = df[["datum", "wegnummer", "zijde"]]

dataset["datum"] = pd.to_datetime(dataset.datum)
dataset["month"] = dataset.datum.dt.month_name()
dataset["day"] = dataset.datum.dt.day_name()
dataset = dataset.drop(columns=["datum"])

dataset["y"] = 1

dataset.head()

/tmp/ipykernel_1478/3700819593.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["datum"] = pd.to_datetime(dataset.datum)
/tmp/ipykernel_1478/3700819593.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["month"] = dataset.datum.dt.month_name()
/tmp/ipykernel_1478/3700819593.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

,wegnummer,zijde,month,day,y
11373,N343,beide,May,Thursday,1
21883,A2,rechts,July,Sunday,1
55158,A28,rechts,January,Saturday,1
52675,N203,beide,May,Thursday,1
84278,A12,rechts,April,Wednesday,1


In [7]:
# zijde
dataset = pd.concat([dataset, dataset.loc[(dataset.y == 1) & (dataset.zijde == "links")].assign(zijde="rechts", y=0)])
dataset = pd.concat([dataset, dataset.loc[(dataset.y == 1) & (dataset.zijde == "rechts")].assign(zijde="links", y=0)])

df.head()

,id,datum,soort_weg,wegnummer,zijde,hm_paal,type_controle,tijd_van_melden,details,laatste_activiteit,...,weer_temp_min,weer_luchtdruk_hpa,weer_luchtvochtigheid_procent,weer_windsnelheid_m_per_sec,weer_windrichting_graden,weer_bewolking_procent,weer_regen_mm,weer_sneeuw_mm,weer_zonnestand,weer_locatie_naam
11373,96207,2022-05-05,nweg,N343,beide,26.6,Radar,10:33:55,\n</td,15:50:05.942842,...,13.99,1024.0,57.0,0.89,202.0,100.0,NaN,NaN,na_zonsopgang,Westerhaar-Vriezenveensewijk
21883,84345,2021-07-11,snelweg,A2,rechts,185.2,Radar,19:45:05,bij afslag budel onder viaduct\n</td,20:10:45.387596,...,20.95,1014.0,60.0,1.54,240.0,81.0,NaN,NaN,na_zonsopgang,Cranendonck
55158,42266,2019-01-12,snelweg,A28,rechts,91.2,Radar,07:43:00,voor afslag zwolle-zuid\r\n,17:15:02.972019,...,6.00,1020.0,87.0,4.60,230.0,90.0,NaN,NaN,voor_zonsopgang,Katerveer
52675,44764,2019-05-09,nweg,N203,beide,54.4,Laser,09:13:00,bij afslag naar de jachthaven vlakbij de a9\n...,11:50:39.597594,...,10.00,995.0,81.0,5.10,180.0,0.0,NaN,NaN,na_zonsopgang,Uitgeest
84278,13094,2017-04-19,snelweg,A12,rechts,65.4,Radar,14:22:41,bij texaco tankstation de forten\r\n,13:45:03.401523,...,8.00,1037.0,45.0,4.60,50.0,20.0,NaN,NaN,na_zonsopgang,Bunnik


In [8]:
negative_records_added = 0

for i, month in enumerate(MONTHS_OF_YEAR):
    remaining_months = MONTHS_OF_YEAR.copy()
    del remaining_months[i]

    records_on_this_month = dataset.loc[(dataset.y == 1) & (dataset.month == month)]
    records_on_this_month_count = len(records_on_this_month)

    print(f"{records_on_this_month_count} occurrences on {month}: adding {records_on_this_month_count * 6} negative records...")
    
    negative_records = []
    for remaining_month in remaining_months:
        negative_records.append(records_on_this_month.assign(month=remaining_month, y=0))
        
    dataset = pd.concat([dataset, *negative_records])

    negative_records_added += records_on_this_month_count * 6

print(f"added {negative_records_added} negative records")

92 occurrences on January: adding 552 negative records...
103 occurrences on February: adding 618 negative records...
99 occurrences on March: adding 594 negative records...
84 occurrences on April: adding 504 negative records...
100 occurrences on May: adding 600 negative records...
80 occurrences on June: adding 480 negative records...
80 occurrences on July: adding 480 negative records...
74 occurrences on August: adding 444 negative records...
77 occurrences on September: adding 462 negative records...
80 occurrences on October: adding 480 negative records...
73 occurrences on November: adding 438 negative records...
58 occurrences on December: adding 348 negative records...
added 6000 negative records


In [9]:
negative_records_added = 0

for i, day in enumerate(DAYS_OF_WEEK):
    remaining_days = DAYS_OF_WEEK.copy()
    del remaining_days[i]
    
    records_on_this_day = dataset.loc[(dataset.y == 1) & (dataset.day == day)]
    records_on_this_day_count = len(records_on_this_day)
    
    print(f"{records_on_this_day_count} occurrences on {day}: adding {records_on_this_day_count * 6} negative records...")
    
    negative_records = []
    for remaining_day in remaining_days:
        negative_records.append(records_on_this_day.assign(day=remaining_day, y=0))
        
    dataset = pd.concat([dataset, *negative_records])
    
    negative_records_added += records_on_this_day_count * 6

print(f"added {negative_records_added} negative records")

143 occurrences on Monday: adding 858 negative records...
211 occurrences on Tuesday: adding 1266 negative records...
185 occurrences on Wednesday: adding 1110 negative records...
175 occurrences on Thursday: adding 1050 negative records...
119 occurrences on Friday: adding 714 negative records...
89 occurrences on Saturday: adding 534 negative records...
78 occurrences on Sunday: adding 468 negative records...
added 6000 negative records


In [10]:
road_numbers = dataset.wegnummer.unique()
negative_records_added = 0

for i, road_number in enumerate(road_numbers):
    remaining_roads = np.delete(road_numbers.copy(), [i])
    remaining_roads_count = len(remaining_roads)
    
    records_on_this_road = dataset.loc[(dataset.y == 1) & (dataset.wegnummer == road_number)]
    records_on_this_road_count = len(records_on_this_road)
    
    print(f"{records_on_this_road_count} occurrences on road {road_number}: adding {records_on_this_road_count * remaining_roads_count} negative records...")
    
    negative_records = []
    for remaining_road in remaining_roads:
        negative_records.append(records_on_this_road.assign(wegnummer=remaining_road, y=0))
        
    dataset = pd.concat([dataset, *negative_records])
        
    negative_records_added += records_on_this_road_count * remaining_roads_count

print(f"added {negative_records_added} negative records")

3 occurrences on road N343: adding 801 negative records...
37 occurrences on road A2: adding 9879 negative records...
26 occurrences on road A28: adding 6942 negative records...
6 occurrences on road N203: adding 1602 negative records...
16 occurrences on road A12: adding 4272 negative records...
5 occurrences on road N212: adding 1335 negative records...
14 occurrences on road A50: adding 3738 negative records...
3 occurrences on road N348: adding 801 negative records...
18 occurrences on road A6: adding 4806 negative records...
7 occurrences on road N344: adding 1869 negative records...
5 occurrences on road N315: adding 1335 negative records...
3 occurrences on road N99: adding 801 negative records...
16 occurrences on road A15: adding 4272 negative records...
6 occurrences on road A67: adding 1602 negative records...
7 occurrences on road N230: adding 1869 negative records...
18 occurrences on road A16: adding 4806 negative records...
1 occurrences on road N231: adding 267 negative

In [11]:
print(dataset.shape)
dataset.head()

df.to_pickle(DATA_DIR / f"prepared_{sample_size}.pkl")

(285407, 5)


One-hot encode columns

In [ ]:
dataset = one_hot_encode(df=dataset, column="zijde")
dataset = one_hot_encode(df=dataset, column="day")
dataset = one_hot_encode(df=dataset, column="month")
dataset = one_hot_encode(df=dataset, column="wegnummer")
# dataset = one_hot_encode(df=dataset, column="type_controle")
# dataset = one_hot_encode(df=dataset, column="hm_paal")

In [181]:
dataset.head()
dataset.shape

(500, 412)

In [200]:
dataset.loc[(dataset.y == 1) & (dataset.day_Monday == True)]

,y,zijde_beide,zijde_links,zijde_rechts,type_controle_Laser,type_controle_Radar,day_Friday,day_Monday,day_Saturday,day_Sunday,...,hm_paal_9.6,hm_paal_9.7,hm_paal_9.8,hm_paal_90.2,hm_paal_90.7,hm_paal_91.5,hm_paal_93.7,hm_paal_95.6,hm_paal_99.0,hm_paal_99.2
0,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,False,False,True,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
389,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
390,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
391,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
